In [1]:
import pandas as pd
import numpy as np
import os
from scipy.cluster import hierarchy as hc
import matplotlib
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from scipy import stats
import math

%matplotlib inline
warnings.filterwarnings(action='ignore')

matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

# Load data

In [3]:
# dir_root = 'C:/Users/kkt71/OneDrive/바탕 화면/기업가치분석'
dir_root = '../../../../../'
dir_master = os.path.join(dir_root,'01. Data','00. master_data')
dir_processed = os.path.join(dir_root, '01. Data','01. 재무지표1', 'ITS')
dir_result = os.path.join(dir_root, '04. 상관분석','01. 재무지표1','02. result')



In [4]:
y = 'EV_EBITDA'
c1 = 'Accenture'
c2 = 'Infosys'
c3 = 'TCS'
c4 = 'Cognizant'

In [5]:
outperform = [c1, c2, c3]
lowperform = [c4]

In [6]:
dir_result_all = os.path.join(dir_result, 'All_ITS', y)

if os.path.exists(dir_result_all)==False:
    os.makedirs(dir_result_all)

# total 기간에 대해서, 존재하는 변수 기준으로 inner join

In [7]:
df_c1 = pd.read_csv(os.path.join(dir_result, c1, y, y+'.csv')).iloc[:,:5]
df_c2 = pd.read_csv(os.path.join(dir_result, c2, y, y+'.csv')).iloc[:,:5]
df_c3 = pd.read_csv(os.path.join(dir_result, c3, y, y+'.csv')).iloc[:,:5]
df_c4 = pd.read_csv(os.path.join(dir_result, c4, y, y+'.csv')).iloc[:,:5]

In [8]:
df_c1.rename({'corr' : df_c1.company.unique()[0]}, axis=1, inplace=True)
df_c2.rename({'corr' : df_c2.company.unique()[0]}, axis=1, inplace=True)
df_c3.rename({'corr' : df_c3.company.unique()[0]}, axis=1, inplace=True)
df_c4.rename({'corr' : df_c4.company.unique()[0]}, axis=1, inplace=True)

df_c1.drop(['company'], axis=1, inplace=True)
df_c2.drop(['company'], axis=1, inplace=True)
df_c3.drop(['company'], axis=1, inplace=True)
df_c4.drop(['company'], axis=1, inplace=True)

In [9]:
df_all_variable = pd.merge(pd.merge(pd.merge(df_c1, df_c2, on=['variable','group', 'importance'], how='inner'), 
                          df_c3, on=['variable','group', 'importance'], how='inner'), 
                 df_c4, on=['variable','group', 'importance'], how='inner')

In [10]:
df_all_variable = df_all_variable[['variable', 'group', 'importance']+outperform+lowperform]

# 0.7 이상 상관계수가 하나라도 있으면, 남기도록 outer join

In [11]:
df_c1 = pd.read_csv(os.path.join(dir_result, c1, y, 'compare_period_corr_07.csv'))
df_c2 = pd.read_csv(os.path.join(dir_result, c2, y, 'compare_period_corr_07.csv'))
df_c3 = pd.read_csv(os.path.join(dir_result, c3, y, 'compare_period_corr_07.csv'))
df_c4 = pd.read_csv(os.path.join(dir_result, c4, y, 'compare_period_corr_07.csv'))

In [12]:
df_c1.rename({'corr_total' : df_c1.company.unique()[0]}, axis=1, inplace=True)
df_c2.rename({'corr_total' : df_c2.company.unique()[0]}, axis=1, inplace=True)
df_c3.rename({'corr_total' : df_c3.company.unique()[0]}, axis=1, inplace=True)
df_c4.rename({'corr_total' : df_c4.company.unique()[0]}, axis=1, inplace=True)

df_c1.drop(['corr_sub', 'company'], axis=1, inplace=True)
df_c2.drop(['corr_sub', 'company'], axis=1, inplace=True)
df_c3.drop(['corr_sub', 'company'], axis=1, inplace=True)
df_c4.drop(['corr_sub', 'company'], axis=1, inplace=True)

In [13]:
df_c1 = df_c1.dropna()
df_c2 = df_c2.dropna()
df_c3 = df_c3.dropna()
df_c4 = df_c4.dropna()

In [14]:
# df_all = pd.merge(pd.merge(df_c1, df_c3, on=['variable','group', 'importance'], how='outer'), 
#                           df_c4, on=['variable','group', 'importance'], how='outer')
                 

In [15]:
df_all = pd.merge(pd.merge(pd.merge(df_c1, df_c2, on=['variable','group', 'importance'], how='outer'), 
                          df_c3, on=['variable','group', 'importance'], how='outer'), 
                 df_c4, on=['variable','group', 'importance'], how='outer')

In [16]:
df_all = df_all[['variable', 'group', 'importance']+outperform+lowperform]

In [17]:
# df_all = df_all[['variable', 'group', 'importance']+[c1, c3, c4]]

# 위의 두 개 inner join

In [18]:
df_all_merged = pd.merge(df_all, df_all_variable, how='inner', on=['variable','group','importance'], suffixes = ("_filtered", "_show"))

In [19]:
# df_all_merged.to_csv(os.path.join(dir_result, 'All_ITS', 'corr_y_x_over7.csv'), index=False, encoding='utf-8-sig')

In [20]:
writer = pd.ExcelWriter(os.path.join(dir_result_all, 'corr_y_x_over7.xlsx'), engine='xlsxwriter')
df_all_merged.to_excel(writer, index=False)
writer.save()

In [21]:
df_all_merged.columns

Index(['variable', 'group', 'importance', 'Accenture_filtered',
       'Infosys_filtered', 'TCS_filtered', 'Cognizant_filtered',
       'Accenture_show', 'Infosys_show', 'TCS_show', 'Cognizant_show'],
      dtype='object')